In [1]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
text = ("After Abraham Lincoln won the November 1860 presidential [MASK] on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces [MASK] Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [4]:
inputs = tokenizer(text, return_tensors='pt')
outputs = model(**inputs)

In [5]:
outputs.keys() # returns MLM output logits

odict_keys(['logits'])

In [6]:
outputs.logits.shape # (1, 46, 30522) - 1 batch, 46 tokens, 30522 vocab size

torch.Size([1, 62, 30522])

In [7]:
# To identify the masked token, we can use the input_ids tensor and find the index of the masked token (103)
mask_position = torch.flatten((inputs.input_ids[0] == 103).nonzero()).tolist()
mask_position # 

[9, 43]

It is for these two positions that we must calculate the loss for when training our model. How does that work? Well, we compare the inputs at those two positions, to the predicted outputs at those two positions - converted to one-hot encoding and probability distribution respectively.

To convert the inputs tokens to one-hot encodings we need the vocab dictionary length, and add a 1 at the position given by the token value.

In [8]:
vocab_size = len(tokenizer.get_vocab())
vocab_size

30522

In [9]:
# Which aligns to our outputs.logits.shape tensor
outputs.logits.shape    

torch.Size([1, 62, 30522])

In [10]:
# In reality, we must mask our tokens randomly, after which we feed the origingal unmasked token ids to the model as labels and keep the token ids as the new masked tensor

In [30]:
# To test this, let's unmask the text first
text = ("After Abraham Lincoln won the November 1860 presidential election on an "
        "anti-slavery platform, an initial seven slave states declared their "
        "secession from the country to form the Confederacy. War broke out in "
        "April 1861 when secessionist forces attacked Fort Sumter in South "
        "Carolina, just over a month after Lincoln's inauguration.")

In [31]:
# Then tokenize it
inputs = tokenizer(text, return_tensors='pt')
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [32]:
inputs.input_ids == 102

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False,  True]])

Now we want to mask a number of tokens in the input_ids tensor, the original BERT model was pretrained using a masking probability of 15%, alongside a few additional rules. We will create a similiar, but simpler implementation by taking the primary 15% masking functionality.

In [33]:
# create a random array of floats in equal dimension to input_ids
rand = torch.rand(inputs.input_ids.shape)
rand

tensor([[0.6651, 0.6721, 0.6261, 0.3289, 0.9107, 0.6470, 0.0555, 0.9104, 0.8449,
         0.6122, 0.4238, 0.7765, 0.2957, 0.3486, 0.6630, 0.9683, 0.4010, 0.0743,
         0.4187, 0.6679, 0.1147, 0.6689, 0.0455, 0.1533, 0.2324, 0.3343, 0.4038,
         0.7323, 0.5335, 0.8741, 0.9388, 0.5298, 0.0506, 0.2529, 0.4933, 0.6202,
         0.5287, 0.1417, 0.5578, 0.8943, 0.5674, 0.3848, 0.7423, 0.5236, 0.6028,
         0.1722, 0.2678, 0.2981, 0.9371, 0.9485, 0.0410, 0.8754, 0.6088, 0.9782,
         0.0732, 0.9573, 0.0460, 0.6160, 0.8543, 0.6093, 0.8623, 0.7028]])

In [34]:
# create a mask tensor where the random array is less than 0.15
mask_arr = (rand < 0.15) * (inputs.input_ids != 102) * (inputs.input_ids != 101)
mask_arr

tensor([[False, False, False, False, False, False,  True, False, False, False,
         False, False, False, False, False, False, False,  True, False, False,
          True, False,  True, False, False, False, False, False, False, False,
         False, False,  True, False, False, False, False,  True, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True, False, False, False,  True, False,  True, False, False, False,
         False, False]])

In [35]:
# extract the indices of the masked tokens
selection = torch.flatten((mask_arr[0]).nonzero()).tolist()
selection

[6, 17, 20, 22, 32, 37, 50, 54, 56]

We then select these True value positions in the input_ids tensor, and change their values to 103 - we need to copy the original tensor first though, to create a new one called labels:

In [36]:
inputs['labels'] = inputs.input_ids.detach().clone()
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,  2281,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,  2019,  3988,  2698,
          6658,  2163,  4161,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,  1012,  2162,  3631,  2041,  1999,  2258,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
          1010,  2074,  2058,  1037,  3204,  2044,  5367,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

And now use mask_arr to mask ~15% of our values:

In [37]:
inputs.input_ids[0, selection] = 103
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,   103,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,   103,  3988,  2698,
           103,  2163,   103,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,   103,  2162,  3631,  2041,  1999,   103,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
           103,  2074,  2058,  1037,   103,  2044,   103,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

In [38]:
inputs.input_ids

tensor([[  101,  2044,  8181,  5367,  2180,  1996,   103,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,   103,  3988,  2698,
           103,  2163,   103,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,   103,  2162,  3631,  2041,  1999,   103,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
           103,  2074,  2058,  1037,   103,  2044,   103,  1005,  1055, 17331,
          1012,   102]])

In [39]:
outputs.loss

In [40]:
inputs

{'input_ids': tensor([[  101,  2044,  8181,  5367,  2180,  1996,   103,  7313,  4883,  2602,
          2006,  2019,  3424,  1011,  8864,  4132,  1010,   103,  3988,  2698,
           103,  2163,   103,  2037, 22965,  2013,  1996,  2406,  2000,  2433,
          1996, 18179,   103,  2162,  3631,  2041,  1999,   103,  6863,  2043,
         22965,  2923,  2749,  4457,  3481,  7680,  3334,  1999,  2148,  3792,
           103,  2074,  2058,  1037,   103,  2044,   103,  1005,  1055, 17331,
          1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([

And run through the same steps again

In [41]:
outputs = model(**inputs)

In [42]:
# And we will find that we now have an additional output tensor, the loss tensor
outputs.keys()

odict_keys(['loss', 'logits'])

In [43]:
outputs.loss

tensor(0.8409, grad_fn=<NllLossBackward0>)

It is this loss we will be training on

In [44]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])